## Interactive Computing with Dynamic Compute - Mosaics

Dynamic Compute is a map computation engine. It enables users to dynamically generate maps from a composable and arbitrary set of Python operations. Together, these properties enable data scientists and the building of complex GIS applications.

In this example notebook we will provide a light overview of the `Dynamic Compute` library through exploring [Kossuth, Iowa's Highest Corn Producing County](https://www.nass.usda.gov/Statistics_by_State/Iowa/Publications/County_Estimates/2021/IA-CtyEst-Corn-02-21.pdf). (I'm sure there's a more compelling use case here this is a starting point)

Before we start we must first import the `Dynamic Compute` client and one of our main image objects the `Mosaic`

In [ ]:
import descarteslabs.dynamic_compute as dc

### Mosaics

We'll start off by creating a `Mosaic` object. (insert better definition of Mosaic here).

Creating a `Mosaic` via its method `from_product_bands` takes a few input arguments:
* [Catalog Product ID:](https://catalog.descarteslabs.com/?/product/airbus:oneatlas:spot:v2) `airbus:oneatlas:spot:v2`
* Bands: `'red green blue nir'`
* Start Date: `'2021-01-01'`
* End Date: `'2022-01-01'`

*Note*: If you are coming from `Workflows`, a `Mosaic` is analogous to an `ImageCollection` with the temporal dimension removed as of current release. We will discuss more advanced temporal analyses in the subsequent `02 Interactive Computing with ImageStacks.ipynb`

In [ ]:
spot_mosaic = dc.Mosaic.from_product_bands(
    "airbus:oneatlas:spot:v2",
    "red green blue nir",
    start_datetime="2021-01-01",
    end_datetime="2022-01-01",
)
type(spot_mosaic)

### Interactive Mapping
Now that we've defined a pointer to the dataset we want to visualize as a `Mosaic`, in this case Airbus SPOT, we can set up an interactive map over which to view our layers. 

We start by:
* Delcaring a `Dynamic Compute map` object
* Setting center coordinates
* Setting a start zoom level

In [ ]:
m = dc.map
m.center = 43.197541, -94.221831
m.zoom = 13

Next let's add our imagery to the map. First we will call `Mosaic.pick_bands` on our `red`, `green`, and `blue` bands, then `Mosaic.visualize` to add to our map viewport with the following arguments:
* Name: `string`
* Map: `Dynamic Compute map` object 
* Scales (_optional_): `list`

In [ ]:
# _ = spot_mosaic.pick_bands('red green blue').visualize('SPOT TCC - Breaks', m)
_ = spot_mosaic.pick_bands("red green blue").visualize(
    "SPOT TCC", m, scales=[[0, 255], [0, 255], [0, 255]]
)

Now we can call our `map` object to view the contents (interactively!):

In [ ]:
m

### RGBs - False Color Composites

We just added the `red`, `green`, and `blue` bands to their respective RGB channels, creating a True Color Composite (TCC). However in the field of remote sensing we are not limited to simply the visible spectrum. Earth Observation professionals regularly switch around which bands correspond to which channel to create [RGB Composites](https://www.nesdis.noaa.gov/news/satellite-imagery-rgbs-adding-value-saving-time), which can tease out visual information that our eyes typically wouldn't see through the visible spectrum. 

In this next cell we will add a standard False Color Composite (FCC) as a new layer, mapping `near-infrared` to `red`, `red` to `green`, and `green` to `blue` visualization channels. This particular RGB composite is [useful for highlighting healthy vegetation](https://earthobservatory.nasa.gov/features/FalseColor/page6.php) in reds/pinks and unhealthy vegetation in duller blues/greens.

In [ ]:
_ = spot_mosaic.pick_bands("nir red green").visualize(
    "SPOT FCC", m, scales=[[0, 255], [0, 255], [0, 255]]
)

Note that the previous `map` updated in real time, we can re-call that same object as many times as we want!

In [ ]:
m

### Band Ratio's - NDVI

Another common practice in remote sensing is creating band ratios. In this next cell we will how to calculate the Normalized Difference in Vegetation Index (NDVI). NDVI typically ranges from -1 to +1, with higher values signifying growing vegetation as plants high in chlorophyll content [reflect more of the near-infrared wavelengths of the electromagnetic spectrum](https://earthobservatory.nasa.gov/features/MeasuringVegetation/measuring_vegetation_2.php).

Our calculation is as follows:

$NDVI = (NIR - RED) / (NIR + RED)$

Here we will use the `Mosaic.unpack_bands` method to reference each band as its own independent variable. We can then visualize the resulting NDVI just as we would any other `Mosaic`!

In [ ]:
nir, red = spot_mosaic.unpack_bands("nir red")
ndvi = (nir - red) / (nir + red)
_ = ndvi.visualize("NDVI", m, colormap="magma", scales=[[-1, 1]])

### Masking
Lastly, we also regularly `mask` one raster layer to another to remove and erroneous pixel values from our visualization or computation. `Dynamic Compute` makes this simple. In this next cell we will mask our input SPOT `Mosaic` to our NDVI `Mosaic` where NDVI is less than 0.3:

In [ ]:
spot_mosaic_masked = spot_mosaic.mask(ndvi < 0.3)
_ = spot_mosaic_masked.pick_bands("nir red green").visualize(
    "FCC - Masked", m, scales=[[0, 255], [0, 255], [0, 255]]
)

In [ ]:
m